In [1]:
import gensim
import os
from tensorflow import keras

In [2]:
import mysql.connector

In [3]:
import re

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [6]:
# load question type classifier
q_type_classifier = keras.models.load_model('model_full.h5')

In [7]:
# load doc2vec embeddings to embed input questions
doc2vec_embeddings = gensim.models.Doc2Vec.load('doc_model_full.h5')

In [8]:
# load knowledge graph database cursor
cnx = mysql.connector.connect(user='nathan', password='3k78LW4e22', host="127.0.0.1", database="ner")
#f = open('ner.sql', 'r')
cursor = cnx.cursor()
#cursor.execute(f.read(), multi=True)

In [9]:
sql_cmd = """SELECT dsynID, name FROM dsynNames;"""
cursor.execute(sql_cmd)

dsynNames = []
for item in cursor:
    dsynNames.append(item)

In [10]:
df = pd.DataFrame(dsynNames, columns=['dsynID', 'label'])

In [28]:
df

,dsynID,label
0,1,active TB
1,1,tus kab mob ntsws tuberculosis
2,1,tb
3,1,kab mob TB
4,1,tus kab mob TB
5,1,latent TB infection
6,1,ltbi
7,1,active TB disease
8,1,pulmonary TB
9,1,khaub thuas


In [12]:
def split_input(input_string):
    data = re.sub('\?', ' ?', input_string)
    return data.split(' ')

In [23]:
input_ = 'Cov tsos mob zoo li cas?'

In [24]:
data = split_input(input_)

In [25]:
classification = np.argmax(q_type_classifier.predict(doc2vec_embeddings.infer_vector(data).reshape(1,32)))

In [26]:
def get_symptom_data(dsynID):
    sql_cmd = """SELECT baseName FROM relationDsynHasSosy
    JOIN sosy ON relationDsynHasSosy.sosyID=sosy.id
    WHERE dsynID={id_};""".format(id_=dsynID)
    cursor.execute(sql_cmd)
    for item in cursor:
        print(item[0])

In [27]:
if classification == 1:
    answer = None
    data_lc = [w.lower() for w in data]
    for item in dsynNames:
        if item[1].lower() in input_.lower():
            answer = item[0]
            break
    if answer == None:
        print("Tus mob no mas nrhiav tsis tau.")
    else:
        print("Tus mob no muaj cov tsos mob hauv qab no:")
        get_symptom_data(answer)
else:
    print("Tsis muaj cov information txog tus mob no.")

Tsis muaj cov information txog tus mob no.
